In [1]:
sc

### Finding the average amount of people using the MTA during different weather conditions during 2011-2016

In [36]:
def mapper1(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    reader = csv.reader(data)
    for row in reader:
        if row[1].split('/')[2] != '2010':
            date = row[1]
            event = row[4]
            numPeople = row[5].split('.')[0]
            if event == 'Normal':
                yield ((date,'Clear Skies'), int(numPeople))
            else:
                yield ((date,event), int(numPeople))
        
def mapper2(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    reader = csv.reader(data)
    for row in reader:
        if row[1].split('/')[2] != '2010':
            date = row[1]
            event = row[4]
            if event == 'Normal':
                yield ((date, 'Clear Skies'), 1)
            else:
                yield ((date, event), 1)

mtaData = sc.textFile('dataset/clean-mta-data/clean-mta-data.csv',use_unicode=False).cache()

# sum up all the entry by weather
# returns (event, total_people)
rdd1 = mtaData.mapPartitionsWithIndex(mapper1) \
                .reduceByKey(lambda x,y: x+y) \
                .map(lambda ((d,e),s): (e,s)) \
                .reduceByKey(lambda x,y: x+y)

# count how many days that have certain weather
# d = date; e = event; c = count;
# returns (event, count)
rdd2 = mtaData.mapPartitionsWithIndex(mapper2) \
               .distinct() \
               .map(lambda ((d,e),c): (e,c)) \
               .reduceByKey(lambda x,y: x+y)

# get average Entry in day for weather
# d = date; e = event; c = count; s = summation
# returns (event, average_peple_per_weather)
rdd3 = rdd1.join(rdd2) \
           .map(lambda (e, (s, c)): (e, s/c)) \
           .collect()
rdd3

[('Thunderstorm', 2311591),
 ('Rain , Snow', 4619726),
 ('Fog , Snow', 3844535),
 ('Clear Skies', 4639989),
 ('Snow', 4354041),
 ('Rain', 4715522),
 ('Fog', 5243310),
 ('Fog , Rain', 4722427),
 ('Fog , Rain , Snow', 4365354)]

### Find the average amount of people using the MTA by day of the week and weather condition

In [37]:
def mapper3(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    import datetime
    reader = csv.reader(data)
    for row in reader:
        if row[1].split('/')[2] != '2010':
            date = datetime.datetime.strptime(row[1], '%m/%d/%Y').strftime("%A")
            event = row[4]
            numPeople = row[5].split('.')[0]
            if event == 'Normal':
                yield ((date, 'Clear Skies'), int(numPeople))
            else:
                yield ((date,event), int(numPeople))
            
def mapper4(index,data):
    # skip header row
    if index==0:
        data.next()
    import csv
    import datetime
    reader = csv.reader(data)
    for row in reader:
        if row[1].split('/')[2] != '2010':
            date = datetime.datetime.strptime(row[1], '%m/%d/%Y').strftime("%A")
            event = row[4]
            if event == 'Normal':
                yield ((date, 'Clear Skies'), 1)
            else:
                yield ((date, event), 1)

import operator
mtaData = sc.textFile('dataset/clean-mta-data/clean-mta-data.csv',use_unicode=False).cache()

# get the amount of people using the mta on each day of the week for different weathers
# returns ((weekday, event), summation_of_passengers)
rdd4 = mtaData.mapPartitionsWithIndex(mapper3)  \
                .reduceByKey(lambda x,y: x+y)

# get the number of days by each day of the week (hour) for different weathers
# returns ((weekday, event), summation_of_day_hour)
rdd5 = mtaData.mapPartitionsWithIndex(mapper4) \
                .distinct() \
                .reduceByKey(lambda x,y: x+y) \
    
rdd6 = rdd4.join(rdd5) \
           .map(lambda (e, (s, c)): (e, s/c)) \
           .sortByKey(True) \
           .collect()

rdd6

KeyboardInterrupt: 